In [1]:
import torch
from vllm import LLM, SamplingParams

model = LLM(model="meta-llama/Llama-2-7b-chat-hf")

/home/ubuntu/.conda/envs/vllm-env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-01-10 01:13:33,007	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


INFO 01-10 01:13:33 llm_engine.py:70] Initializing an LLM engine with config: model='meta-llama/Llama-2-7b-chat-hf', tokenizer='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=4096, download_dir=None, load_format=auto, tensor_parallel_size=1, quantization=None, enforce_eager=False, seed=0)
INFO 01-10 01:13:39 llm_engine.py:275] # GPU blocks: 815, # CPU blocks: 512
INFO 01-10 01:13:41 model_runner.py:501] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 01-10 01:13:41 model_runner.py:505] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode.
INFO 01-10 01:13:49 model_runner.py:547] Graph capturing finished in 8 secs.


In [62]:
params = SamplingParams(max_tokens=32)
prompts = ["In a galaxy far, far away, a"]

outputs = model.generate(prompts, params)

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  1.03it/s]


In [66]:
print(outputs[0].outputs[0].text)
print(outputs[0].outputs[0].token_ids)
print(outputs[0].prompt_token_ids)

 grumpy old Mandalorian named Karga helps a young adventurer named Carlos on his quest to find a rare artifact. But when they are confront
[867, 398, 2272, 2030, 15419, 284, 272, 713, 4257, 476, 21899, 6911, 263, 4123, 17623, 9945, 4257, 10815, 373, 670, 21126, 304, 1284, 263, 10812, 24238, 29889, 1205, 746, 896, 526, 21751]
[1, 512, 263, 15400, 29891, 2215, 29892, 2215, 3448, 29892, 263]


In [35]:
import time
def get_decode_tput(batch_size=1, num_tokens=256):
    prompts = ["Hello"] * batch_size
    sampling_params = SamplingParams(max_tokens=num_tokens, ignore_eos=True)

    start = time.perf_counter()
    outputs = model.generate(prompts, sampling_params)
    torch.cuda.synchronize()
    end = time.perf_counter()

    total_time = end - start
    total_tokens = 0
    for output in outputs:
        total_tokens += len(output.outputs[0].token_ids)

    print(f"----- Batch Size: {batch_size} -----")
    print(f"Total time: {total_time:0.2f}s")
    print(f"Total tokens: {total_tokens} tokens")
    print(f"Tput: {total_tokens / total_time:0.2f} tokens/sec")

In [36]:
batch_sizes = [1,2,4,8,16,32,64]
NUM_TOKENS = 256

for batch_size in batch_sizes:
    get_decode_tput(batch_size, NUM_TOKENS)

Processed prompts: 100%|██████████| 1/1 [00:07<00:00,  7.75s/it]


----- Batch Size: 1 -----
Total time: 7.76s
Total tokens: 256 tokens
Tput: 33.01 tokens/sec


Processed prompts: 100%|██████████| 2/2 [00:07<00:00,  3.87s/it]


----- Batch Size: 2 -----
Total time: 7.74s
Total tokens: 512 tokens
Tput: 66.15 tokens/sec


Processed prompts: 100%|██████████| 4/4 [00:07<00:00,  1.97s/it]


...

----- Batch Size: 16 -----
Total time: 8.84s
Total tokens: 4096 tokens
Tput: 463.17 tokens/sec


Processed prompts: 100%|██████████| 32/32 [00:10<00:00,  3.10it/s]


----- Batch Size: 32 -----
Total time: 10.33s
Total tokens: 8192 tokens
Tput: 792.81 tokens/sec


Processed prompts: 100%|██████████| 64/64 [00:15<00:00,  4.02it/s]

----- Batch Size: 64 -----
Total time: 15.94s
Total tokens: 16384 tokens
Tput: 1027.81 tokens/sec


In [44]:
def get_prefill_tput(num_words, iterations=10):
    prompts = ("Hello_" * num_words)[:-1]
    sampling_params = SamplingParams(max_tokens=1)

    start = time.perf_counter()
    for _ in range(iterations):
        outputs = model.generate(prompts, sampling_params)
    torch.cuda.synchronize()
    end = time.perf_counter()

    total_time = end - start
    prompt_tokens = len(outputs[0].prompt_token_ids)
    total_tokens = prompt_tokens * iterations

    print(f"\n----- Prompt Tokens: {prompt_tokens} -----")
    print(f"Total time: {total_time:0.2f}s")
    print(f"Total tokens: {total_tokens} tokens")
    print(f"Tput: {total_tokens / total_time:0.2f} tokens/sec\n")

In [45]:
num_words_list = [16, 64, 128, 256, 512, 1024, 2048]

for num_words in num_words_list:
    get_prefill_tput(num_words, iterations=10)

Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 28.85it/s]



----- Prompt Tokens: 32 -----
Total time: 0.37s
Total tokens: 320 tokens
Tput: 874.93 tokens/sec



Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 24.42it/s]



----- Prompt Tokens: 128 -----
Total time: 0.43s
Total tokens: 1280 tokens
Tput: 2977.90 tokens/sec



Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 14.00it/s]


...


----- Prompt Tokens: 1024 -----
Total time: 2.67s
Total tokens: 10240 tokens
Tput: 3841.92 tokens/sec



Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  2.02it/s]



----- Prompt Tokens: 2048 -----
Total time: 4.98s
Total tokens: 20480 tokens
Tput: 4116.02 tokens/sec



Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  1.03it/s]


----- Prompt Tokens: 4096 -----
Total time: 9.78s
Total tokens: 40960 tokens
Tput: 4188.37 tokens/sec

